# Analyse results for query dissemination experiment

In [19]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 500)

# 1. Parse Logfiles

In [20]:
paths = {
    "10": "../../logs/query-dissemination/29-03-2021/selectivity_10",
    "25": "../../logs/query-dissemination/29-03-2021/selectivity_25",
    "80": "../../logs/query-dissemination/29-03-2021/selectivity_80"
}

num_nodes = 73 # number of nodes per run

num_runs = {
    10:3,
    25:3,
    80:3
}

In [21]:
# The function stores the lat and lon values for each selectivity.
def determine_values(selectivity, idx):
    selectivity_to_values = {
        10:(90.0,0.0),
        25:(50.0, 50.0),
        80:(20.0,0.0)        
    }
    val = selectivity_to_values[selectivity]
    return val[idx]

In [22]:
sels = []
categories = []
node_ids = []
lats = []
lons = []
runs = []
connect_outliers = []

received = []
runs_rec = []
sels_received = []
query_times = []

startup_times = {}

for selectivity in paths:
    for root, dirs, files in os.walk(paths[selectivity]):
        #print("Root: " + root)
        run = root.split("/")[-1].split("_")[-1]            
        for file in files :
            #print("File: " + file)
            n_id = file[:6]  
            if n_id[:4] != 'base' :       
                # get node_id
                if n_id[-1:] == '_' :
                    node_id = int(n_id[4:5])                
                else :
                    node_id = int(n_id[4:6])
            #print(node_id)
            with open(os.path.join(root, file)) as log:             
                connect_attempts = 0
                query_time = 100000

                for line in log :            
                    log_time = int(line.split(" ")[0])
                    elem = line.split( )                

                    if "initial attribute range" in line: 
                        tmp = str(elem[-1:])
                        t = tmp[3:-3]                    
                        coords = t.split(',')
                        lat = float(coords[0])
                        lon = float(coords[1])
                        
                        node_ids.append( node_id)
                        lats.append(lat)
                        lons.append(lon)
                        runs.append(run)
                        sels.append(selectivity)
                    if "! RECEIVED QUERY" in line :
                        received.append(int(node_id))
                        runs_rec.append(run)     
                        sels_received.append(selectivity)

                        query_time = elem[8]
                        query_times.append(query_time)
                    elif "STARTUP TIME" in line:
                        elem = line.split( )
                        startup_times[(selectivity, run)] = int(elem[-1])
                    # sanity checks
                    if "ACTOR DOWN" in line and log_time < query_time and time < 59000:
                        print("Actor down in: selectivity " + selectivity + ", run " + str(run) + ", node" + str(node_id) + ", query time: " 
                                + str(query_time) + ", log_time: " + str(log_time))     
                    if "RECONNECTING TO INITIAL PARENT" in line:
                        print("Initial connection issues in: selectivity " + selectivity + ", run " + str(run) + ", node" + str(node_id))     
                    if "could not reach" in line:                        
                        print("Found Error in: selectivity " + selectivity + ", run " + run + ", node" + node_id) 
                
                if (connect_attempts > 5):
                    m_connect_outliers.append("run " + run + " node " + node_id)
            
#print(received)
#print('node_ids ' + str(len(node_ids)))
#print('categories ' + str(len(categories)))
#print('lat ' + str(len(lats)))
#print('lon ' + str(len(lons)))
#print(len(sels))

initial_ranges_tmp = pd.DataFrame(np.column_stack([sels,runs,node_ids, lats, lons]), columns=['selectivity','run', 'node_id','lat', 'lon'])

initial_ranges = initial_ranges_tmp.astype({
    'selectivity':'int32',
    'run':'int32',
    'node_id':'int32',
    'lat':'float',
    'lon':'float'
})

print(initial_ranges.dtypes)
initial_ranges

selectivity      int32
run              int32
node_id          int32
lat            float64
lon            float64
dtype: object


,selectivity,run,node_id,lat,lon
0,10,2,3,54.629926,78.552097
1,10,2,48,86.991263,95.038275
2,10,2,40,36.215096,45.616298
3,10,2,21,1.907784,95.965673
4,10,2,59,12.370182,79.426896
...,...,...,...,...,...
652,80,1,69,50.133969,18.371031
653,80,1,17,72.837441,6.926584
654,80,1,66,46.430967,66.578599
655,80,1,47,81.882704,70.319071


## Outliers &  Sanity checks

In [23]:
print("Nodes with connection issues:")
connect_outliers

Nodes with connection issues:


[]

In [24]:
nan = initial_ranges[initial_ranges.isna().any(axis=1)]
nan

,selectivity,run,node_id,lat,lon


In [25]:
nul = initial_ranges[initial_ranges.isnull().any(axis=1)]
nul

,selectivity,run,node_id,lat,lon


# 2. Determine which nodes received the query
For all selectivities, create a df that contains all nodes that received the query in each run

In [26]:
received_tmp = pd.DataFrame(np.column_stack([sels_received, runs_rec, received, query_times]), 
                            columns=['selectivity','run', 'node_id', 'time_received'])
received = received_tmp.astype({
    'selectivity':'int64',
    'run':'int64',
    'node_id':'int64',
    'time_received':'int64'
})
received['time_received'] = received.apply(lambda row: row['time_received'] - startup_times[(str(row['selectivity']),str(row['run']))], axis=1)

received.head()

,selectivity,run,node_id,time_received
0,10,2,10,25086715489
1,10,2,30,25087269487
2,10,2,69,25087452570
3,10,2,54,25087785176
4,10,2,0,25085662189
5,10,2,61,25087329879
6,10,2,9,25087362870
7,10,2,8,25087490836
8,10,2,6,25087796774
9,10,2,1,25086718005


In [27]:
received['time_received'].max()

25502099490

# 3. Delete all nodes from the df that did not match the query

In [28]:
nodes_applying = initial_ranges[
    (initial_ranges['lat'] >= initial_ranges.apply(lambda x: determine_values(x['selectivity'],0), axis=1) ) &
    (initial_ranges['lon'] >= initial_ranges.apply(lambda x: determine_values(x['selectivity'],1), axis=1) ) 
]
nodes_applying.groupby(['selectivity']).size().reset_index(name='applies_to')


,selectivity,applies_to
0,10,27
1,25,65
2,80,166


### Sanity check

Find the nodes where the query applied according to (lat,lon), but did not receive it

In [29]:
a_waldo = nodes_applying.merge(received, how='left', on=['selectivity', 'run', 'node_id'])
len(a_waldo)

258

In [30]:
w = a_waldo[a_waldo.isnull().any(axis=1)]
w.head()

,selectivity,run,node_id,lat,lon,time_received


# 4. Count the nodes that the respective query applies to in each run for every selectivity

In [31]:
grouped = nodes_applying.groupby(['selectivity','run']).size().reset_index(name='applies_to')
print(grouped.dtypes)
grouped

selectivity    int64
run            int64
applies_to     int64
dtype: object


,selectivity,run,applies_to
0,10,0,8
1,10,1,10
2,10,2,9
3,25,0,26
4,25,1,22
5,25,2,17
6,80,0,51
7,80,1,57
8,80,2,58


# 5. Count the nodes that received the query in each run for every selectivity

In [32]:
received_grouped = received.groupby(['selectivity','run']).size().reset_index(name='num_received')
print(received_grouped.dtypes)
received_grouped

selectivity     int64
run             int64
num_received    int64
dtype: object


,selectivity,run,num_received
0,10,0,13
1,10,1,13
2,10,2,12
3,25,0,28
4,25,1,23
5,25,2,18
6,80,0,53
7,80,1,59
8,80,2,60


# 6. Compute Overhead etc.

### Table columns
- applies_to            = number of nodes where the query applies 
- received              = number of nodes that received the query
- overhead              = received - applies
- baseline_ovehead      = num_nodes - applies
- num_nodes             = topology size
- overhead_ratio        = overhead/baseline_overhead = ratio
- overhead_ratio_pct             = overhead_ratio * 100

In [33]:
overview_tmp = grouped.merge(received_grouped, how='left', on=['selectivity', 'run'])#.astype({'num_received':'int64'})
print(overview_tmp.dtypes)
overview_tmp

selectivity     int64
run             int64
applies_to      int64
num_received    int64
dtype: object


,selectivity,run,applies_to,num_received
0,10,0,8,13
1,10,1,10,13
2,10,2,9,12
3,25,0,26,28
4,25,1,22,23
5,25,2,17,18
6,80,0,51,53
7,80,1,57,59
8,80,2,58,60


In [34]:
overview = overview_tmp

overview['rime_overhead_abs'] = overview.apply(lambda row: row.num_received - row.applies_to, axis=1 )
overview['num_nodes'] = num_nodes
overview['baseline_overhead'] = overview.apply(lambda row: num_nodes - row.applies_to, axis=1)
overview['overhead_ratio'] = overview.apply(lambda row: row.rime_overhead_abs/row.baseline_overhead, axis=1)
overview['overhead_ratio_pct'] = overview.apply(lambda row: row.overhead_ratio*100, axis=1)

print(overview.dtypes)
overview

selectivity             int64
run                     int64
applies_to              int64
num_received            int64
rime_overhead_abs       int64
num_nodes               int64
baseline_overhead       int64
overhead_ratio        float64
overhead_ratio_pct    float64
dtype: object


,selectivity,run,applies_to,num_received,rime_overhead_abs,num_nodes,baseline_overhead,overhead_ratio,overhead_ratio_pct
0,10,0,8,13,5,73,65,0.076923,7.692308
1,10,1,10,13,3,73,63,0.047619,4.761905
2,10,2,9,12,3,73,64,0.046875,4.687500
3,25,0,26,28,2,73,47,0.042553,4.255319
4,25,1,22,23,1,73,51,0.019608,1.960784
5,25,2,17,18,1,73,56,0.017857,1.785714
6,80,0,51,53,2,73,22,0.090909,9.090909
7,80,1,57,59,2,73,16,0.125000,12.500000
8,80,2,58,60,2,73,15,0.133333,13.333333


In [35]:
summary = pd.DataFrame()

summary['mean_overhead_ratio']      = overview.groupby('selectivity')['overhead_ratio_pct'].mean()
summary['median_overhead_ratio']    = overview.groupby('selectivity')['overhead_ratio_pct'].median()
summary['applies_pct']              = overview.groupby('selectivity')['applies_to'].mean() / num_nodes
summary['received_pct']             = overview.groupby('selectivity')['num_received'].mean() / num_nodes

summary['base_overhead']            = 1 - summary['applies_pct']
summary['rime_overhead']            = summary['received_pct'] - summary['applies_pct']
summary['overhead_saved_by_rime']   = (summary['base_overhead'] - summary['rime_overhead']) * 100

In [36]:
summary

,mean_overhead_ratio,median_overhead_ratio,applies_pct,received_pct,base_overhead,rime_overhead,overhead_saved_by_rime
selectivity,,,,,,,
10,5.713904,4.761905,0.123288,0.173516,0.876712,0.050228,82.648402
25,2.667273,1.960784,0.296804,0.315068,0.703196,0.018265,68.493151
80,11.641414,12.500000,0.757991,0.785388,0.242009,0.027397,21.461187
